あらゆる情報をまずは表にして，それから可視化する

- 

In [1]:
import json 
import pandas as pd

In [2]:
# data_dir_path = "/Users/keisukeonoue/ws/lukasiewicz_2/experiments"
result_file_path_list = [
    "./experiments/version_101/result.json",
    "./experiments/version_102/result.json",
    "./experiments/version_103/result.json",
    "./experiments/version_104/result.json",
    "./experiments/version_105/result.json",
    "./experiments/version_106/result.json",
]

In [3]:
result_file_path = result_file_path_list[0]

with open(result_file_path, 'r') as f:
    json_data = json.load(f)

json_data

{'path': './experiments/version_101',
 'source_paths': ['data/pima_indian_diabetes/diabetes_cleaned_normalized.csv',
  'data/pima_indian_diabetes/diabetes_discretized.csv'],
 'experiment_name': 'pima_indian_diabetes_cv_10',
 'seed': 42,
 'n_splits': 5,
 'n_unsupervised': 15,
 'c1': 10,
 'c2': 10,
 'rule_thr': 0,
 'result': {'fold_0': {'RuleFit Classifier (disc)': {'accuracy': 0.7761194029850746,
    'precision': 0.7333333333333333,
    'recall': 0.5,
    'f1': 0.5945945945945945,
    'auc': 0.8353535353535353,
    'violation_detail': {'0': [79,
      79,
      [1,
       3,
       5,
       6,
       7,
       9,
       11,
       13,
       14,
       21,
       25,
       26,
       29,
       32,
       33,
       37,
       38,
       40,
       41,
       45,
       47,
       48,
       51,
       54,
       55,
       56,
       57,
       58,
       61,
       62,
       65,
       66,
       68,
       69,
       71,
       72,
       73,
       74,
       76,
       77,
     

In [4]:
for i in json_data:
    print(i)

print()

for i in json_data['result']:
    print(i)

print()

for i in json_data['result']['fold_0']:
    print(i)

path
source_paths
experiment_name
seed
n_splits
n_unsupervised
c1
c2
rule_thr
result

fold_0
fold_1
fold_2
fold_3
fold_4

RuleFit Classifier (disc)
tree generator (disc)
RuleFit Classifier (conti)
tree generator (conti)
linear svm (L)
non-linear svm (L)
logistic regression (L)
linear svm
non-linear svm
logistic regression
rule_based_prediction


In [5]:
for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']:
    print(i)

print()

for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']['violation_detail']:
    print(i)

print()

for i in json_data['result']['fold_0']['RuleFit Classifier (disc)']['violation_detail']['0']:
    print(i)

accuracy
precision
recall
f1
auc
violation_detail
n_violation
n_rule
violation_rate
n_violation (instance)
n_evaluation (instance)
violation_rate (instance)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33

79
79
[1, 3, 5, 6, 7, 9, 11, 13, 14, 21, 25, 26, 29, 32, 33, 37, 38, 40, 41, 45, 47, 48, 51, 54, 55, 56, 57, 58, 61, 62, 65, 66, 68, 69, 71, 72, 73, 74, 76, 77, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 96, 99, 100, 102, 103, 104, 105, 106, 107, 108, 110, 112, 113, 114, 115, 116, 118, 120, 122, 125, 126, 127, 129, 132, 133]


## 一時保存した実験結果の読み込み

In [6]:
result_dfs = []
infos = []

for path in result_file_path_list:
    with open(path, 'r') as f:
        json_data = json.load(f)
    
    infos.append(json_data['rule_thr'])
    
    tmp = []

    for fold, _ in json_data['result'].items():
        result_df = pd.DataFrame(json_data['result'][fold]).T.drop(['violation_detail'], axis=1)
        tmp.append(result_df)

    result_dfs.append(tmp)

print(len(result_dfs))
print(len(result_dfs[0]))
print()

model_name_list = [
    'RuleFit Classifier (disc)', 
    'tree generator (disc)', 
    'RuleFit Classifier (conti)', 
    'tree generator (conti)', 
    'linear svm (L)', 
    'non-linear svm (L)', 
    'logistic regression (L)', 
    'linear svm', 
    'non-linear svm', 
    'logistic regression', 
    'rule_based_prediction'
]

combined_dfs = [pd.concat(dfs, ignore_index=False) for dfs in result_dfs]
dfs_mean = [combined_df.groupby(combined_df.index).mean().reindex(index=model_name_list) for combined_df in combined_dfs]
dfs_std = [combined_df.groupby(combined_df.index).std().reindex(index=model_name_list) for combined_df in combined_dfs]

print(len(dfs_mean))

6
5

6


## 実験結果の保存（クロスバリデーション）

In [7]:
for version, (dfs, info) in enumerate(zip(result_dfs, infos)):
    
    for nth_fold, df in enumerate(dfs):

        df['nth_fold'] = nth_fold
        df['n_splits'] = 5
        df['C1'] = 10
        df['C2'] = 10
        df['n_unsupervised'] = 15
        df['rule_thr'] = info

        file_path = f'./experiment_10/fold_{nth_fold}/ver_{version}.csv'
        print(file_path)

        df.to_csv(file_path)


./experiment_10/fold_0/ver_0.csv
./experiment_10/fold_1/ver_0.csv
./experiment_10/fold_2/ver_0.csv
./experiment_10/fold_3/ver_0.csv
./experiment_10/fold_4/ver_0.csv
./experiment_10/fold_0/ver_1.csv
./experiment_10/fold_1/ver_1.csv
./experiment_10/fold_2/ver_1.csv
./experiment_10/fold_3/ver_1.csv
./experiment_10/fold_4/ver_1.csv
./experiment_10/fold_0/ver_2.csv
./experiment_10/fold_1/ver_2.csv
./experiment_10/fold_2/ver_2.csv
./experiment_10/fold_3/ver_2.csv
./experiment_10/fold_4/ver_2.csv
./experiment_10/fold_0/ver_3.csv
./experiment_10/fold_1/ver_3.csv
./experiment_10/fold_2/ver_3.csv
./experiment_10/fold_3/ver_3.csv
./experiment_10/fold_4/ver_3.csv
./experiment_10/fold_0/ver_4.csv
./experiment_10/fold_1/ver_4.csv
./experiment_10/fold_2/ver_4.csv
./experiment_10/fold_3/ver_4.csv
./experiment_10/fold_4/ver_4.csv
./experiment_10/fold_0/ver_5.csv
./experiment_10/fold_1/ver_5.csv
./experiment_10/fold_2/ver_5.csv
./experiment_10/fold_3/ver_5.csv
./experiment_10/fold_4/ver_5.csv


## 実験結果の保存（クロスバリデーション，平均と標準偏差）

In [8]:
for version, (info, df) in enumerate(zip(infos, dfs_mean)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = f'./experiment_10/ver_{version}_mean.csv'
    print(file_path)

    df.to_csv(file_path)

for version, (info, df) in enumerate(zip(infos, dfs_std)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = f'./experiment_10/ver_{version}_std.csv'
    print(file_path)

    df.to_csv(file_path)

./experiment_10/ver_0_mean.csv
./experiment_10/ver_1_mean.csv
./experiment_10/ver_2_mean.csv
./experiment_10/ver_3_mean.csv
./experiment_10/ver_4_mean.csv
./experiment_10/ver_5_mean.csv
./experiment_10/ver_0_std.csv
./experiment_10/ver_1_std.csv
./experiment_10/ver_2_std.csv
./experiment_10/ver_3_std.csv
./experiment_10/ver_4_std.csv
./experiment_10/ver_5_std.csv


## ルールの整形と保存（csv）

In [9]:
for nth_fold in range(5):
    for version in range(len(infos)):
        file_path_1 = f"./experiments/version_10{version + 1}/rules/rules_{nth_fold}.csv"
        file_path_2 = f"./experiments/version_10{version + 1}/rules/rules_{nth_fold}.txt"

        file_path_3 = f"./experiments/version_10{version + 1}/result.json"
        with open(file_path_3, 'r') as f:
            result = json.load(f)
        rule_result = result['result'][f'fold_{nth_fold}']['rule_based_prediction']['violation_detail']
        df_rule_result = pd.DataFrame(rule_result).T

        
        df = pd.read_csv(file_path_1, index_col=0)

        KB = []
        with open(file_path_2, 'r') as f:
            for line in f:
                KB.append(line.rstrip())

        df.drop(['type', 'support', 'importance'], axis=1, inplace=True)
        df['rule'] = KB[:df.shape[0]]

        n_predicate_list = [] 
        implication_bool_list = []
        for h, rule in enumerate(df['rule']):
            n_predicate_list.append(sum([1 for element in rule.split() if element not in ['⊗', '¬', '→']]))

            # consequent カラムのために後で追加
            if rule[rule.split().index('→') + 1] == '¬':
                implication_bool_list.append('¬')
            else:
                implication_bool_list.append(None)

        df['n_predicate'] = n_predicate_list
        df['rank'] = df['coef'].apply(lambda x: sum(abs(x) > info for info in infos) - 1)


        df['n_violation (instance)'] = df_rule_result[1].to_list()
        df['n_evaluation (instance)'] = df_rule_result[0].to_list()
        df['violation'] = df['n_violation (instance)'].apply(lambda x: 0 if x == 0 else 1)
        df['consequent'] = implication_bool_list

        display(df.head(3))

        file_path = f'./experiment_10/fold_{nth_fold}/rules_{version}.csv'
        df.to_csv(file_path)


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
0,Pregnancies_Low → ¬ Outcome,-0.353076,2,1,79,79,1,None
3,Glucose_Low → ¬ Outcome,-1.237307,2,5,31,31,1,None
5,Glucose_High → Outcome,1.367137,2,5,0,26,0,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
0,Pregnancies_Low → ¬ Outcome,-0.353076,2,1,79,79,1,None
3,Glucose_Low → ¬ Outcome,-1.237307,2,5,31,31,1,None
5,Glucose_High → Outcome,1.367137,2,5,0,26,0,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.237307,2,5,31,31,1,None
5,Glucose_High → Outcome,1.367137,2,5,0,26,0,None
12,BMI_Low → ¬ Outcome,-0.664089,2,3,45,45,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.237307,2,5,31,31,1,None
5,Glucose_High → Outcome,1.367137,2,5,0,26,0,None
12,BMI_Low → ¬ Outcome,-0.664089,2,3,45,45,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.237307,2,5,31,31,1,None
5,Glucose_High → Outcome,1.367137,2,5,0,26,0,None


...

,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.091643,2,5,22,22,1,None
5,Glucose_High → Outcome,1.217328,2,5,0,29,0,None
12,BMI_Low → ¬ Outcome,-0.989185,2,4,40,40,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.091643,2,5,22,22,1,None
5,Glucose_High → Outcome,1.217328,2,5,0,29,0,None
12,BMI_Low → ¬ Outcome,-0.989185,2,4,40,40,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.091643,2,5,22,22,1,None
5,Glucose_High → Outcome,1.217328,2,5,0,29,0,None
12,BMI_Low → ¬ Outcome,-0.989185,2,4,40,40,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.091643,2,5,22,22,1,None
5,Glucose_High → Outcome,1.217328,2,5,0,29,0,None
12,BMI_Low → ¬ Outcome,-0.989185,2,4,40,40,1,None


,rule,coef,n_predicate,rank,n_violation (instance),n_evaluation (instance),violation,consequent
3,Glucose_Low → ¬ Outcome,-1.091643,2,5,22,22,1,None
5,Glucose_High → Outcome,1.217328,2,5,0,29,0,None


In [10]:
# # モデルごとに予測が違

# for version in range(len(infos)):
#     file_path = f"./experiments/version_10{version + 1}/result.json"

#     with open(file_path, 'r') as f:
#         result = json.load(f)
    
#     result['result'][f'fold_{nth_fold}']

In [7]:
result_dfs[0][0]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.776119,0.733333,0.5,0.594595,0.835354,27,34,0.794118,526,603,0.872305
tree generator (disc),0.776119,0.675,0.613636,0.642857,0.817045,23,34,0.676471,516,603,0.855721
RuleFit Classifier (conti),0.776119,0.684211,0.590909,0.634146,0.810101,29,34,0.852941,529,603,0.87728
tree generator (conti),0.776119,0.684211,0.590909,0.634146,0.834722,28,34,0.823529,526,603,0.872305
linear svm (L),0.328358,0.328358,1.0,0.494382,0.212374,19,34,0.558824,449,603,0.74461
non-linear svm (L),0.69403,0.526316,0.681818,0.594059,0.783586,26,34,0.764706,180,603,0.298507
logistic regression (L),0.768657,0.709677,0.5,0.586667,0.793939,23,34,0.676471,136,603,0.225539
linear svm,0.768657,0.69697,0.522727,0.597403,0.809596,30,34,0.882353,530,603,0.878939
non-linear svm,0.80597,0.821429,0.522727,0.638889,0.816667,30,34,0.882353,537,603,0.890547
logistic regression,0.776119,0.71875,0.522727,0.605263,0.821465,30,34,0.882353,533,603,0.883914


In [8]:
result_dfs[5][0]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.776119,0.733333,0.5,0.594595,0.835354,2,2,1.0,32,57,0.561404
tree generator (disc),0.776119,0.675,0.613636,0.642857,0.817045,2,2,1.0,36,57,0.631579
RuleFit Classifier (conti),0.776119,0.684211,0.590909,0.634146,0.810101,2,2,1.0,38,57,0.666667
tree generator (conti),0.776119,0.684211,0.590909,0.634146,0.834722,2,2,1.0,35,57,0.614035
linear svm (L),0.671642,0.5,0.863636,0.633333,0.804545,2,2,1.0,6,57,0.105263
non-linear svm (L),0.671642,0.5,0.681818,0.576923,0.794444,1,2,0.5,2,57,0.035088
logistic regression (L),0.753731,0.666667,0.5,0.571429,0.787879,1,2,0.5,2,57,0.035088
linear svm,0.768657,0.69697,0.522727,0.597403,0.809596,2,2,1.0,32,57,0.561404
non-linear svm,0.80597,0.821429,0.522727,0.638889,0.816667,2,2,1.0,35,57,0.614035
logistic regression,0.776119,0.71875,0.522727,0.605263,0.821465,2,2,1.0,33,57,0.578947


In [9]:
result_dfs[5][1]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.731343,0.709677,0.44898,0.55,0.736375,2,3,0.666667,24,64,0.375
tree generator (disc),0.656716,0.529412,0.55102,0.54,0.702041,2,3,0.666667,24,64,0.375
RuleFit Classifier (conti),0.664179,0.545455,0.489796,0.516129,0.739496,3,3,1.0,29,64,0.453125
tree generator (conti),0.738806,0.675,0.55102,0.606742,0.765186,3,3,1.0,29,64,0.453125
linear svm (L),0.365672,0.365672,1.0,0.535519,0.366627,1,3,0.333333,21,64,0.328125
non-linear svm (L),0.679104,0.548387,0.693878,0.612613,0.782713,1,3,0.333333,1,64,0.015625
logistic regression (L),0.686567,0.612903,0.387755,0.475,0.765426,2,3,0.666667,9,64,0.140625
linear svm,0.708955,0.647059,0.44898,0.53012,0.786074,3,3,1.0,30,64,0.46875
non-linear svm,0.716418,0.677419,0.428571,0.525,0.776951,3,3,1.0,30,64,0.46875
logistic regression,0.716418,0.666667,0.44898,0.536585,0.781513,3,3,1.0,30,64,0.46875


In [10]:
result_dfs[5][2]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.798507,0.827586,0.521739,0.64,0.828187,1,1,1.0,1,27,0.037037
tree generator (disc),0.761194,0.733333,0.478261,0.578947,0.787055,1,1,1.0,10,27,0.37037
RuleFit Classifier (conti),0.761194,0.684211,0.565217,0.619048,0.802125,1,1,1.0,3,27,0.111111
tree generator (conti),0.798507,0.827586,0.521739,0.64,0.837698,1,1,1.0,5,27,0.185185
linear svm (L),0.343284,0.343284,1.0,0.511111,0.211709,0,1,0.0,0,27,0.0
non-linear svm (L),0.738806,0.59322,0.76087,0.666667,0.824358,0,1,0.0,0,27,0.0
logistic regression (L),0.783582,0.814815,0.478261,0.60274,0.813982,1,1,1.0,6,27,0.222222
linear svm,0.791045,0.78125,0.543478,0.641026,0.826334,1,1,1.0,2,27,0.074074
non-linear svm,0.791045,0.8,0.521739,0.631579,0.824605,1,1,1.0,4,27,0.148148
logistic regression,0.798507,0.806452,0.543478,0.649351,0.824852,1,1,1.0,3,27,0.111111


In [11]:
result_dfs[5][3]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.850746,0.714286,0.625,0.666667,0.87546,1,2,0.5,32,53,0.603774
tree generator (disc),0.776119,0.525,0.65625,0.583333,0.8125,2,2,1.0,33,53,0.622642
RuleFit Classifier (conti),0.761194,0.5,0.625,0.555556,0.815564,2,2,1.0,36,53,0.679245
tree generator (conti),0.835821,0.647059,0.6875,0.666667,0.861366,2,2,1.0,34,53,0.641509
linear svm (L),0.604478,0.356164,0.8125,0.495238,0.718137,2,2,1.0,16,53,0.301887
non-linear svm (L),0.768657,0.509091,0.875,0.643678,0.882353,1,2,0.5,1,53,0.018868
logistic regression (L),0.835821,0.666667,0.625,0.645161,0.889093,1,2,0.5,3,53,0.056604
linear svm,0.858209,0.709677,0.6875,0.698413,0.898591,1,2,0.5,32,53,0.603774
non-linear svm,0.835821,0.727273,0.5,0.592593,0.901961,2,2,1.0,36,53,0.679245
logistic regression,0.843284,0.689655,0.625,0.655738,0.903186,2,2,1.0,34,53,0.641509


In [12]:
result_dfs[5][4]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.761194,0.727273,0.510638,0.6,0.804598,2,2,1.0,27,51,0.529412
tree generator (disc),0.738806,0.676471,0.489362,0.567901,0.742969,2,2,1.0,28,51,0.54902
RuleFit Classifier (conti),0.731343,0.648649,0.510638,0.571429,0.817804,2,2,1.0,29,51,0.568627
tree generator (conti),0.761194,0.741935,0.489362,0.589744,0.864025,2,2,1.0,30,51,0.588235
linear svm (L),0.350746,0.350746,1.0,0.519337,0.251162,1,2,0.5,22,51,0.431373
non-linear svm (L),0.731343,0.584615,0.808511,0.678571,0.850819,1,2,0.5,2,51,0.039216
logistic regression (L),0.783582,0.821429,0.489362,0.613333,0.866226,1,2,0.5,7,51,0.137255
linear svm,0.768657,0.722222,0.553191,0.626506,0.868427,2,2,1.0,26,51,0.509804
non-linear svm,0.753731,0.769231,0.425532,0.547945,0.870139,2,2,1.0,32,51,0.627451
logistic regression,0.776119,0.774194,0.510638,0.615385,0.871851,2,2,1.0,31,51,0.607843


In [13]:
dfs_mean[0]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,29.0,36.0,0.798061,517.6,590.6,0.87455
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,26.8,36.0,0.738482,507.6,590.6,0.857428
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,31.6,36.0,0.880583,519.4,590.6,0.878731
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,31.4,36.0,0.873556,523.0,590.6,0.884506
linear svm (L),0.541791,0.434768,0.856103,0.559086,0.562332,23.4,36.0,0.651444,293.8,590.6,0.474153
non-linear svm (L),0.71791,0.547521,0.761836,0.635432,0.821875,23.8,36.0,0.658986,150.8,590.6,0.256131
logistic regression (L),0.770149,0.732412,0.470891,0.570208,0.826615,19.6,36.0,0.549053,95.4,590.6,0.162218
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,31.4,36.0,0.874652,521.0,590.6,0.881041
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,32.4,36.0,0.900467,528.8,590.6,0.894698
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,31.6,36.0,0.879916,523.6,590.6,0.88567


In [14]:
dfs_mean[1]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,14.2,18.2,0.777647,360.2,403.0,0.883547
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,13.8,18.2,0.757647,361.0,403.0,0.883918
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,16.4,18.2,0.904248,366.6,403.0,0.901585
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,16.0,18.2,0.880915,366.8,403.0,0.902848
linear svm (L),0.534328,0.42329,0.882062,0.559399,0.667946,11.6,18.2,0.635882,217.2,403.0,0.52167
non-linear svm (L),0.701493,0.530627,0.785606,0.627725,0.827467,13.2,18.2,0.72366,111.2,403.0,0.272393
logistic regression (L),0.776119,0.757705,0.470891,0.577723,0.825458,10.0,18.2,0.552092,46.4,403.0,0.120501
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,16.2,18.2,0.892026,365.0,403.0,0.897846
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,16.6,18.2,0.913137,368.8,403.0,0.907298
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,16.4,18.2,0.903137,366.8,403.0,0.902507


In [15]:
dfs_mean[2]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,8.0,10.6,0.762179,167.2,203.4,0.816573
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,8.2,10.6,0.787179,169.4,203.4,0.825449
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,10.0,10.6,0.95,173.2,203.4,0.847277
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,9.6,10.6,0.909615,173.0,203.4,0.842546
linear svm (L),0.325373,0.325373,1.0,0.489178,0.348506,6.8,10.6,0.648077,164.0,203.4,0.799575
non-linear svm (L),0.714925,0.544039,0.759575,0.631353,0.827801,6.4,10.6,0.605128,40.8,203.4,0.190773
logistic regression (L),0.765672,0.71091,0.487739,0.573507,0.825165,6.0,10.6,0.537179,24.6,203.4,0.113695
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,9.2,10.6,0.859615,170.6,203.4,0.830717
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,9.6,10.6,0.901282,174.0,203.4,0.849608
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,9.4,10.6,0.884615,172.4,203.4,0.840436


In [16]:
dfs_mean[3]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,3.4,4.4,0.783333,65.8,95.2,0.677642
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,3.6,4.4,0.883333,68.8,95.2,0.715086
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,4.4,4.4,1.0,70.4,95.2,0.727428
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,4.2,4.4,0.966667,70.0,95.2,0.721755
linear svm (L),0.410448,0.353204,0.9625,0.515872,0.554638,2.8,4.4,0.675,59.0,95.2,0.577074
non-linear svm (L),0.725373,0.561531,0.738021,0.633911,0.826788,2.4,4.4,0.55,11.0,95.2,0.10462
logistic regression (L),0.768657,0.716496,0.496076,0.581533,0.825359,2.8,4.4,0.666667,10.6,95.2,0.108118
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,4.0,4.4,0.866667,67.8,95.2,0.695724
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,4.2,4.4,0.966667,70.8,95.2,0.731554
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,4.2,4.4,0.966667,69.6,95.2,0.716067


In [17]:
dfs_mean[4]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,2.2,3.0,0.78,39.8,68.8,0.500069
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,2.4,3.0,0.88,42.8,68.8,0.586544
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,3.0,3.0,1.0,44.2,68.8,0.570246
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,2.8,3.0,0.96,43.6,68.8,0.567118
linear svm (L),0.508955,0.401706,0.914819,0.553183,0.546588,2.0,3.0,0.64,22.0,68.8,0.254421
non-linear svm (L),0.723881,0.555985,0.75976,0.639835,0.826895,1.2,3.0,0.36,5.4,68.8,0.057221
logistic regression (L),0.768657,0.716496,0.496076,0.581533,0.825061,2.0,3.0,0.68,8.0,68.8,0.118951
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,2.6,3.0,0.86,41.4,68.8,0.520691
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,2.8,3.0,0.96,44.4,68.8,0.573127
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,2.8,3.0,0.96,43.2,68.8,0.549155


In [18]:
dfs_mean[5]

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815994,1.6,2.0,0.833333,23.2,50.4,0.421325
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,1.8,2.0,0.933333,26.2,50.4,0.509722
RuleFit Classifier (conti),0.738806,0.612505,0.556312,0.579261,0.797018,2.0,2.0,1.0,27.0,50.4,0.495755
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,2.0,2.0,1.0,26.6,50.4,0.496418
linear svm (L),0.467164,0.383173,0.935227,0.538908,0.470436,1.2,2.0,0.566667,13.0,50.4,0.233329
non-linear svm (L),0.71791,0.547063,0.764015,0.63569,0.826937,0.8,2.0,0.366667,1.2,50.4,0.021759
logistic regression (L),0.768657,0.716496,0.496076,0.581533,0.824521,1.2,2.0,0.633333,5.4,50.4,0.118359
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,1.8,2.0,0.9,24.4,50.4,0.443561
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,2.0,2.0,1.0,27.4,50.4,0.507526
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,2.0,2.0,1.0,26.2,50.4,0.481632
